In [11]:
from NetworkStructure import Generator
from NetworkStructure import Discriminator
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse.linalg import eigsh
from torch.autograd import Variable
from torch.functional import Tensor
from torchvision import transforms
from torch import nn
import torch
import torchextractor as tx
import torchvision
%matplotlib inline
import torch

In [16]:
# parameters
image_size = 28
img_shape = 1,28,28
latent_dim = 100
image_channel = 1
batch_size = 1

In [17]:
data_transform = transforms.Compose(
    [
        transforms.CenterCrop(28),
        transforms.ToTensor(),
        transforms.Normalize(0.5, 0.5),
    ]
)

train_dataset = torchvision.datasets.MNIST("./data", train=True, download=True,transform=data_transform)
# test_dataset = torchvision.datasets.MNIST("./data", train=False,download=True,transform=data_transform)
# min_test = torch.utils.data.Subset(test_dataset, torch.arange(1000))
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size= batch_size,
    num_workers=2,
    shuffle=True,
    drop_last=True,
)

# train_data = enumerate(train_dataloader)
# a = next(train_data)
# a[1][0].shape
# enumerate = tuple(0,data) tuple(1,data) tuple(number,data)
# iter = list[data]
# data = list[data,label]
train_data = iter(train_dataloader)
# a = next(train_data)
# a[0].shape

In [18]:
PATH_G = "./pre-train/checkpoint389G"
PATH_D = "./pre-train/checkpoint389D"
checkpoint_G = torch.load(PATH_G)
checkpoint_D = torch.load(PATH_D)
model_G = Generator(image_size, latent_dim, image_channel)
model_D = Discriminator(image_channel)
model_G.load_state_dict(checkpoint_G['model_state_dict']) # print every layer name
model_D.load_state_dict(checkpoint_D['model_state_dict'])
model_G.eval()
model_D.eval()

Discriminator(
  (model): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Dropout2d(p=0.25, inplace=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Dropout2d(p=0.25, inplace=False)
    (6): BatchNorm2d(32, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): LeakyReLU(negative_slope=0.2, inplace=True)
    (9): Dropout2d(p=0.25, inplace=False)
    (10): BatchNorm2d(64, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (11): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (12): LeakyReLU(negative_slope=0.2, inplace=True)
    (13): Dropout2d(p=0.25, inplace=False)
    (14): BatchNorm2d(128, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
  )
  (adv_laye

In [19]:
# Function using to calculate Lapula matrix
def Lapula(feature, number, nor=False):
    """
    Calculate lapula metrix
    :param feature: input the feature
    :param number:  the number of feature
    :param nor:  default is False, if true Lapula matrix will be normalize
    :return:  lapura or sys lapula
    """
    # 1. Calcute the Similarity matrix and Degree matrix
    similarity_martix = np.empty((number,number),dtype=float)
    d_matrix = np.zeros((number,number))
    for i in range(feature.shape[0]):
        x1 = feature[i]
        for r  in range(i,feature.shape[0]):
            if i == r:
                similarity_martix[i][r] = 0
            else:
                x2 = feature[r]
                weight = torch.nn.functional.cosine_similarity(x1,x2,0).detach().numpy()
                # print(weight)
                if weight >= 0.7:
                    d_matrix[i][i] += 1
                    similarity_martix[i][r]= 1-weight
                    similarity_martix[r][i]= 1-weight
                else:
                    similarity_martix[i][r]= 0
                    similarity_martix[r][i]= 0
    lapula = d_matrix - similarity_martix
    if nor==True:
         t = np.sum(d_matrix, axis= 1)
         V = np.diag(t**(-0.5))
         where_are_inf = np.isinf(V)
         V[where_are_inf] = 0
         lapula = V@lapula@V

    return lapula

In [26]:
data = next(train_data)
data[0].shape, data[1].shape

(torch.Size([1, 1, 28, 28]), torch.Size([1]))